# <center> Практика. EDA + Feature Engineering. Соревнование на Kaggle </center>

### **Постановка задачи:**

**ИССЛЕДОВАНИЕ ДАННЫХ:**

✍ В этом модуле вы будете работать с датасетом, в котором содержатся сведения о 515 000 отзывов на отели Европы. Модель, которую вы будете обучать, должна предсказывать рейтинг отеля по данным сайта Booking на основе имеющихся в датасете данных.

***ДАННЫЕ САЙТА BOOKING:***

Наименование столбца:  | Описание столбца:
------- | --------
***hotel_address***   | адрес отеля;
***review_date***   | дата, когда рецензент разместил соответствующий отзыв;
***average_score***   | средний балл отеля, рассчитанный на основе последнего комментария за последний год;
***hotel_name***   | название отеля;
***reviewer_nationality***   | страна рецензента;
***negative_review***   | отрицательный отзыв, который рецензент дал отелю;
***review_total_negative_word_counts***   | общее количество слов в отрицательном отзыв;
***positive_review***   | положительный отзыв, который рецензент дал отелю;
***review_total_positive_word_counts***   | общее количество слов в положительном отзыве.
***reviewer_score***   | оценка, которую рецензент поставил отелю на основе своего опыта;
***total_number_of_reviews_reviewer_has_given***   | количество отзывов, которые рецензенты дали в прошлом;
***total_number_of_reviews***   | общее количество действительных отзывов об отеле;
***tags***   | теги, которые рецензент дал отелю;
***days_since_review***   | количество дней между датой проверки и датой очистки;
***additional_number_of_scoring***   | есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
***lat***   | географическая широта отеля;
***lng***   | географическая долгота отеля.

Содержание работы

1. Импорт библиотек и загрузка данных.
2. Очистка данных.
3. Исследование данных.
4. Генерация признаков.
5. Преобразование признаков.
6. Отбор признаков.
7. Обучение модели.

## **1. Импорт библиотек и загрузка данных:**

In [1]:
# Загружаем необходимые библиотеки:
import pandas as pd
import numpy as np

from scipy import stats
import statsmodels.api as sm
from statsmodels import stats as sms

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

from geopy.distance import geodesic as GD 

# Делаем визуальную настройку графиков:
sns.set_theme("notebook") 
sns.set_palette("Set2")

In [18]:
# Загружаем датасет:
hotels = pd.read_csv('Data\hotels.csv')
hotels.head(3)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643


1.1. Выведем на экран сводную информацию методом info():

In [3]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

**Вывод:** в датасете 386803 записей (есть пропущенные значения).

## **2. Очистка данных:**

2.1. Проверим данные на наличие дубликатов и удалим найденные дубликаты:

In [4]:
# Проверяем наличие дубликатов:
print('Количество дубликатов: {}'.format(hotels[hotels.duplicated()].shape[0]))

# Удаляем дубликаты:
# hotels = hotels.drop_duplicates()
# print('Размер таблицы после удаления дубликатов: {}'.format(hotels.shape))

Количество дубликатов: 307
Размер таблицы после удаления дубликатов: (386496, 17)


**Вывод:** представленные данные содержат 307 дубликатов.

## **3. Исследование данных:**

3.1. Выведем на экран основные статистические характеристики
данных по каждому числовому признаку:

In [5]:
hotels.describe(include = 'all')

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
count,386496,386496.000000,386496,386496.000000,386496,386496,386496,386496.000000,386496.000000,386496,386496.000000,386496.000000,386496.000000,386496,386496,384048.000000,384048.000000
unique,1493,NaN,731,NaN,1492,225,248828,NaN,NaN,311737,NaN,NaN,NaN,47135,731,NaN,NaN
top,163 Marsh Wall Docklands Tower Hamlets London ...,NaN,8/2/2017,NaN,Britannia International Hotel Canary Wharf,United Kingdom,No Negative,NaN,NaN,No Positive,NaN,NaN,NaN,"[' Leisure trip ', ' Couple ', ' Double Room '...",1 days,NaN,NaN
freq,3587,NaN,1910,NaN,3587,183952,95833,NaN,NaN,26863,NaN,NaN,NaN,3853,1910,NaN,NaN
mean,NaN,498.504375,NaN,8.397421,NaN,NaN,NaN,18.539400,2744.679231,NaN,17.778163,7.176211,8.397299,NaN,NaN,49.443988,2.823740
std,NaN,500.365093,NaN,0.547861,NaN,NaN,NaN,29.703569,2316.934876,NaN,21.724766,11.052453,1.635747,NaN,NaN,3.468266,4.580782
min,NaN,1.000000,NaN,5.200000,NaN,NaN,NaN,0.000000,43.000000,NaN,0.000000,1.000000,2.500000,NaN,NaN,41.328376,-0.369758
25%,NaN,169.000000,NaN,8.100000,NaN,NaN,NaN,2.000000,1161.000000,NaN,5.000000,1.000000,7.500000,NaN,NaN,48.214277,-0.143649
50%,NaN,342.000000,NaN,8.400000,NaN,NaN,NaN,9.000000,2134.000000,NaN,11.000000,3.000000,8.800000,NaN,NaN,51.500198,-0.000250
75%,NaN,660.000000,NaN,8.800000,NaN,NaN,NaN,23.000000,3633.000000,NaN,22.000000,8.000000,9.600000,NaN,NaN,51.516384,4.834443


3.2. Заполняем пропуски:

In [6]:
hotels = hotels.fillna(0)

## **4. Генерация признаков:**

4.1. Иследуем признак hotel_address и создадим новые признаки 'country' и 'city':

In [7]:
# Создание нового признака 'country':

hotels['country'] = hotels['hotel_address'].apply(lambda x: x.split()[-1] 
        if x.split()[-1] != 'Kingdom' 
        else ' '.join(x.split()[-2:]))

print('\n В датасете представлены отели из '+ str(hotels['country'].nunique()) + ' стран.\n')
print(hotels['country'].value_counts())


 В датасете представлены отели из 6 стран.

country
United Kingdom    196773
Spain              45132
France             44528
Netherlands        43004
Austria            29177
Italy              27882
Name: count, dtype: int64


In [8]:
# Создание нового признака 'city':

hotels['city'] = hotels.apply(lambda x: x['hotel_address'].split()[-5] 
        if x['country'] == 'United Kingdom'
        else x['hotel_address'].split()[-2], axis=1)

print('\n В датасете представлены отели из '+ str(hotels['city'].nunique()) + ' городов.\n')
print(hotels['city'].value_counts())


 В датасете представлены отели из 6 городов.

city
London       196773
Barcelona     45132
Paris         44528
Amsterdam     43004
Vienna        29177
Milan         27882
Name: count, dtype: int64


4.2. Приведем дату отзыва к формату datetime:

In [19]:
hotels['review_date'] = pd.to_datetime(hotels['review_date'])

# выделим год и месяц из даты отзыва
hotels['review_year'] = hotels['review_date'].dt.year
hotels['review_month'] = hotels['review_date'].dt.month
hotels.head()

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng,review_year,review_month
0,Stratton Street Mayfair Westminster Borough Lo...,581,2016-02-19,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671,2016,2
1,130 134 Southampton Row Camden London WC1B 5AF...,299,2017-01-12,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097,2017,1
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,2016-10-18,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643,2016,10
3,216 Avenue Jean Jaures 19th arr 75019 Paris Fr...,34,2015-09-22,7.5,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,607,Friendly staff quiet comfortable room spotles...,11,8,10.0,"[' Leisure trip ', ' Solo traveler ', ' Standa...",681 day,48.888697,2.394540,2015,9
4,Molenwerf 1 1014 AG Amsterdam Netherlands,914,2016-03-05,8.5,Golden Tulip Amsterdam West,Poland,Torn sheets,4,7586,The staff was very friendly and helpful Break...,20,10,9.6,"[' Business trip ', ' Couple ', ' Standard Dou...",516 day,52.385601,4.847060,2016,3


## **5. Преобразование признаков:**


## **6. Отбор признаков:**

6.1. Определим в данных неинформативные признаки, которые не будут участвовать в исследовании:

In [11]:
# Удаляем столбцы из DataFrame:
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
hotels.drop(object_columns, axis = 1, inplace=True)

## **7. Обучение модели:**

In [12]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score'] 

In [13]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

In [14]:
# Наборы данных с меткой "train" будут использоваться для обучения
# модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [15]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor
# инструмент для создания и обучения модели  
from sklearn import metrics
# инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга
# отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)

In [16]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test),
# и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE)
# и показывает среднюю абсолютную процентную ошибку
# предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.1417257126350131


## **Итоговые результаты исследования:**